# FDTD tidy3d

gdsfactory simulation plugin for tidy3d
[tidy3D is a FDTD web based software](https://simulation.cloud/)

## Materials


In [ ]:
# basic ipython configuration (reload source code automatically and plots inline)
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import tidy3d as td
from tidy3d import web
import gdsfactory as gf
import gdsfactory.simulation.gtidy3d as gt

In [ ]:
nm = 1e-3
wavelength = np.linspace(1500, 1600)*nm
f = td.C_0/wavelength

n = gt.materials.material_library['cSi']['Li1993_293K'].eps_model(f)
plt.plot(wavelength, np.abs(n)**0.5)
plt.title('cSi crystalline silicon')
plt.xlabel('wavelength')
plt.ylabel('n')

In [ ]:
n = gt.materials.material_library['SiN']['Horiba'].eps_model(f)
plt.plot(wavelength, np.abs(n)**0.5)
plt.title('SiN')
plt.xlabel('wavelength')
plt.ylabel('n')

In [ ]:
n = gt.materials.material_library['SiO2']['Horiba'].eps_model(f)
plt.plot(wavelength, np.abs(n)**0.5)
plt.title('SiO2')
plt.xlabel('wavelength')
plt.ylabel('n')

## Simulations

In [ ]:
c = gf.components.mmi1x2()
sim = gt.get_simulation(c)
fig = gt.plot_simulation(sim)

In [ ]:
c = gf.components.coupler_ring()
sim = gt.get_simulation(c)
fig = gt.plot_simulation(sim)

In [ ]:
c = gf.components.bend_circular(radius=2)
sim = gt.get_simulation(c)
fig = gt.plot_simulation(sim)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tidy3d as td
from tidy3d import web
import gdsfactory as gf
import gdsfactory.simulation.gtidy3d as gt

c = gf.components.straight_rib(length=3)
sim = gt.get_simulation(c, plot_modes=True)
fig = gt.plot_simulation_xz(sim)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tidy3d as td
from tidy3d import web
import gdsfactory as gf
import gdsfactory.simulation.gtidy3d as gt
from gdsfactory.components.taper import taper_sc_nc

c = taper_sc_nc(length=10)
sim = gt.get_simulation(c, plot_modes=True)
fig = gt.plot_simulation_xz(sim)

In [ ]:
components =  ['bend_euler', 'bend_s', 'coupler', 'coupler_ring', 'crossing', 'mmi1x2', 'mmi2x2', 'taper',  'straight']

for component_name in components:
    print(component_name)
    plt.figure()
    c = gf.components.factory[component_name]()
    sim = gt.get_simulation(c)
    fig = gt.plot_simulation(sim)


### Sparameters

We can run one simulation as well as a group of simulations in paralell

```python
c = gf.components.straight()
s = gt.get_sparameters(c)
```

```python
for component_name, component_factory in gf.components._factory_passives.items():
    c = component_factory()
    s = gt.get_sparameters(c)
    
```    

### simulation results

Once simulations finish we can compute:

- Sparameters (transmission, Reflection)
- Coupling coefficients